# Automate Boring Tasks With Your Own Functions
* Автор статьи: [Fábio Neves](https://github.com/fnneves)
* Автор перевода: [Антон Неверович](https://github.com/AntonNeverovich)
* Оригинал статьи: [towardsdatascience.com](https://towardsdatascience.com/automate-boring-tasks-with-your-own-functions-a32785437179)

***

# Автоматизация рутинных задач с помощью собственных функций
<p>Статья о том как важно для аналитика данных автоматизировать свои рутинные задачи, писать функции, которые будут использоваться из проекта в проект. В статье есть несколько примеров таких функций, например, для объединения csv-файлов или переименования колонок датафреймов.</p>

![Because repetition is usually boring!](scr/img/2020-01-16_001.jpeg)

Автоматизировать задачи с помощью Python очень просто. После того, как у вас есть скрипт, который работает, превратите его в функцию, которая поможет вам писать код более эффективно!

Независимо от того, являетесь ли вы новичком в Python или профессионалом, используете ли вы Python на работе или дома, скорее всего, у вас также есть определенные задачи, которые вы должны выполнять несколько раз в день/неделю/месяц. Вы возвращаетесь к старым проектам, пытаясь найти тот удивительный кусок кода, который вы разработали для другого проекта? — Где же он? Я могу поклясться, что сделал это точно в другом проекте! — ... знакомая мысль?

<i><blockquote style='font-size:130%'> Эта "проблема" может быть особенно актуальна, если, как и я, вы много извлекаете, преобразовываете и загружаете (ETL) данных.</blockquote></i>

Есть несколько примеров, которые я могу предложить в этой статье, и я уверен, что вы можете найти сотни других статей с крутыми трюками и функциями, некоторые лучше, а некоторые хуже. Я тоже их видел. Но большинство из них покажет вам снова одни и те же фрагменты.

Именно поэтому я попытался обойти типичные примеры и поделиться тем, что я действительно создал и регулярно использую! Это не самые удивительные и хорошо написанные функции, но, по крайней мере, я уверен, что вы не видели их в других местах, и, возможно, вы даже можете применить одну или две, чтобы помочь себе в будущем. В следующей статье я покажу, как создать свой собственный класс в Python.

***

### О каких задачах пойдет речь?
Есть ли у вас папка, в которую вы загружаете csv-файлы каждый день, а затем время от времени возникает необходимость объединять их в один большой набор данных? Или, может быть, вы хотите добавить один столбец к каждому файлу в определенной папке и сохранить их с другим именем? Возможно, данные, которые вы извлекаете из определеной базы данных, имеют странные имена столбцов, или один из столбцов должен быть разделен на два?

***У Вас уже есть идеи?!***

Я собираюсь поделиться некоторыми фрагментами кода для решения этих проблем! Ниже приведен список типов задач, которые мы будем кодировать в этой статье. Я надеюсь, что некоторые из них так же полезны и для вас в вашей ежедневной работе!


1. Объединение всех файлов в определенной папке
2. Редактирование файлов в одной папке и их сохранение их
3. Очистка заголовка ваших наборов данных
4. Разбитие столбцов фрейма данных на два или более
5. Фильтрация определенных столбцов фрейма данных по их именам
6. Вычисление количества дней между двумя датами
7. Вычисление количества недель/месяцев/лет между двумя датами

***

### Обработка csv файлов
Я помню, что был очень горд, когда получил эти две функции для работы. Несколько минут (или даже часов) волшебным образом превратились в несколько секунд. "Это мелочи в жизни", наверное!

Имея дело с большим количеством csv-файлов в течение дня, я ежедневно извлекал данные, которые нуждались в дальнейшем форматировании. Если у вас есть похожие задачи, то эти функции вам наверняка понравятся!

#### 1. Объединение всех файлов в определенной папке
Решение днной задачи начиналось как простая функция, но затем немного эволюционировала. Отправной точкой было просто указать на папку с файлами csv, которые я хотел объединить, и сохранить объединенный набор данных в новом файле. Обратите внимание, что я использую файлы с точно такой же структурой, но вы можете легко адаптировать этот код в соответствии с вашими потребностями.

Но csv-файлы могут вмещать большое количество строк, поэтому через некоторое время я добавил опцию слияния файлов только в памяти. И я добавил входную переменную, так что я мог просто вызвать функцию, и она спросила бы у меня путь, вместо того, чтобы я копировал путь, когда я писал саму команду.

In [2]:
import glob
import pandas as pd
from time import strftime

def folder_csv_merge(file_prefix, folder_path='', memory='no'):
    """
    file_prefix: if you want to add a prefix to the name of final merged file
    folder_path: no need to declare it. string copied from file explorer to the folder where the files are
    """
    if folder_path == '':
        folder_path = input('Please enter the path where the CSV files are:\n')
    folder_path = folder_path.replace("\\","/")
    if folder_path[:-1] != "/":
        folder_path = folder_path + "/"

    file_list = glob.glob(folder_path + '*.csv')

    combined = pd.concat( [ pd.read_csv(f) for f in file_list ] )
    if memory == 'no':
        combined.to_csv(folder_path + 'combined_{}_{}.csv'.format(file_prefix, strftime("%Y%m%d-%H%M%S")), index=False)
    else:
        return combined
    print('done')

Вам просто нужно импортировать `glob` и `pandas` для этого. `Strftime` - это просто хороший способ отслеживать, когда файл был создан, добавляя дату к имени файла.

По умолчанию вы можете просто запустить эту функцию следующим образом: `folder_csv_merge('my_prefix')`. Затем он предложит вам вставить путь к файлу. Новый файл должен появиться в той же папке, что и оригиналы. Это может быть не лучшим решением, поэтому не забудьте отредактировать последнюю строку, чтобы она соответствовала местоположению, в котором вы хотите сохранить окончательные файлы.

Если вы просто хотите получить все данные из csv-файлов во фрейм данных в памяти, а не в новый файл, просто добавьте аргумент `memory='yes'`, но затем вам нужно вызвать функцию в переменную. Например: `merged_file = folder_csv_merge ('my_prefix', memory = 'yes')`. Теперь у вас будет переменная со всеми файлами, объединенными внутри.

### 2. Редактирование нескольких файлов одновременно
Недавно я понял, что провел несколько часов в ожидании извлечения сотен файлов, но забыл добавить важную колонку к каждому из них во время извлечения. Ждать еще один день, чтобы иметь все снова не было вариантом.…
Никаких проблем! Вот для чего нужны циклы ... !

In [3]:
import glob
import pandas as pd

def mass_edit(file_prefix, folder_path=''):
    """
    file_prefix: string that defines new file name
    folder_path: no need to declare it. string copied from file explorer to the folder where the files are
    """
    if folder_path == '':
        folder_path = input('Please enter the path where the CSV files are:\n')
    folder_path = folder_path.replace("\\","/")
    if folder_path[:-1] != "/":
        folder_path = folder_path + "/"

    file_list = glob.glob(folder_path + '*.csv')

    for file in file_list:
        name_pos = file.rfind('\\')
        data = pd.read_csv(file)
        # changes go here!!
        data['seniority'] = data['seniority'] + 1 #in this case, i just needed to add 1 to this column in each file
        # until here!!
        data.to_csv(folder_path + file[name_pos+1:], index=False) #saving the file again with same name
        print(file+' ready!!')

Первая часть функции очень похожа на предыдущую, потому что я хотел, чтобы консоль запрашивала у меня путь к файлу. После указания пути к файлу будет запущен цикл для каждого файла.

В приведенном выше примере у меня была колонка в каждом файле, которая показывала, сколько дней клиент был в нашей компании. Проблема в том, что я забыл добавить `1` к этому значению при извлечении и преобразовании (потому что `seniority = 0` выглядело странно). Это было единственное изменение, которое мне нужно было сделать, но вы можете добавить больше функционала, если хотите. Переименовывайте столбцы, удаляйте столбцы, создавайте новые и т. д.
Это сэкономило много времени!

***

### Преобразование фреймов данных в соответствии с вашими потребностями
Следующие функции — это очистка тех наборов данных, которые вы получаете от отдела продаж или других коллег, которые просто не могут заботиться о согласованности и удобочитаемости данных! Даже некоторые базы данных, построенные давным-давно, могут иметь странные имена столбцов, которые выглядят как предложения, заполненные странными символами.
#### 3. Очистка заголовков фреймов данных
Это довольно легко и весьма полезно. В значительной степени само собой разумеющееся! Это также важно, поскольку необходимо сохранять имена столбцов в соответствии с определенными проектами.

In [4]:
def clean_header(df):
    """
    This functions removes weird characters and spaces from column names, while keeping everything lower case
    """
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

### 4. Разбить столбец на основе его содержимого
Далее, простая функция, которая поможет вам разделить столбец на основе определенного символа или строки.

In [5]:
def split_columns(df, orig, one, two, separator):
    """
    orig: string name of the column we want to split
    one: string name of the first new column
    two: string name of the second new column
    separator: string - this is the character(s) that will define where to split
    """
    df[[one,two]] = df[orig].str.split(separator,expand=True)

Допустим, у вас есть столбец `'tag'` в фрейме данных, который на самом деле состоит из категории и подкатегории, разделенных символом подчеркивания. Вы бы предпочли, чтобы эти две части были разделены, что означает, что нам нужен столбец с именем `'category'` и другой с именем `'subcategory'`.

Сделать это можно достаточно легко:

`split_columns(df=data_frame, orig='tag', one='category', two='subcategory', separator='_')`

Проверьте фрейм данных еще раз, и теперь у вас есть два новых столбца!

### 5. Фильтрация определенныч столбцов фрейма данных по их именам
Это не то, что я использую каждый день, но есть определенный файл, который заставил меня создать его. Этот файл содержит несколько столбцов с одинаковыми именами для разных лет. Но обычно мне просто нужно работать с одним годом, поэтому я должен фильтровать соответствующие столбцы.

In [6]:
def df_filter(df, string):
    new = df.filter(regex=string)
    return new

# Example
data = [{'2018_inc': 10, '2019_inc': 12, '2018_rev':23}, {'2018_inc':10, '2019_inc': 20, '2018_rev': 30}]
df = pd.DataFrame(data)
y_2018 = df_filter(df, '2018') # getting all columns with 2018 in their names

Я включил небольшой пример в конце этого фрагмента, чтобы сделать его более ясным! Просто запустите его и убедитесь сами, как он работает.

***

## Манипуляции с датами
Когда я начал изучать Python, я помню, что одной из самых крутых функций, которые я нашел в то время, была встроенная функция даты/времени. Мы все были нубами в какой-то момент! Эти следующие функции предполагают, что вы уже преобразовали столбцы даты в объект datetime. Это можно сделать используя библиотечную функцию: 

`pd.to_datetime(df['col_name'])`.

#### 6. Вычисление количества дней между двумя датами
Я буду первым, кто признает, что эта следующая функция не является величайшим прорывом всех времен, если только столбцы, которые вы хотите использовать, всегда одни и те же, и меняется только имя фрейма данных. Если это так, вы можете ***захардкодить имена столбцов в функции*** и использовать их с одним аргументом — то есть именем фрейма данных.

Эта функция создает новый столбец с разницей между двумя датами.

In [8]:
def days_diff(df):
    df['CohortIndex_d'] = (df['last_active_date'] - df['signup_date']).dt.days_diff

### 7. Вычисление количества недель/месяцев/лет между датами
Нужно было рассчитать временные ряды. Дневные ряды могут быть получены с помощью функции выше, но она не будет работать для месячных или годовых рядов.

Первая функция `get_date_it()` необходима, чтобы мы могли отделить год, месяц и неделю от каждой даты. Функция возвращает целое число для каждой переменной, и мы включим его в функцию `calc_cohorts()`.

In [9]:
def get_date_int(df, date_column):
    year = df[date_column].dt.year
    month = df[date_column].dt.month
    week = df[date_column].dt.week
    return year, month, week

def calc_cohorts(df, signup_date, last_active):
    """
    signup_date: name of the column with the signup date
    last_active: name of the column with the last login
    """
    sign_year, sign_month, sign_week = get_date_int(df, signup_date)
    lastlogin_year, lastlogin_month, lastlogin_week = get_date_int(df, last_active)
    years_diff = lastlogin_year - sign_year
    months_diff = lastlogin_month - sign_month
    weeks_diff = lastlogin_week - sign_week
    df['cohort_W'] = years_diff * 52 + weeks_diff + 1
    df['cohort_M'] = years_diff * 12 + months_diff + 1

Переходя ко второй функции, нам нужен фрейм данных с двумя столбцами даты (в данном случае это дата регистрации и дата последнего входа в систему, но вы можете изменить имена в соответствии с вашим проектом).
Как только вы запустите его, вы получите два новых столбца в вашем фрейме данных, называемых `cohort_W` и `cohort_M`.